In [35]:
# Read stocks
import yfinance as yf

# To get analysts info
import yahoo_fin.stock_info as si
import yahoo_fin.news as news
import yahoo_fin.options as options

# Required to calculate net present value
import numpy_financial as npf

# For reading properties
from jproperties import Properties

import pandas as pd

In [3]:
TICKER = 'AAPL'
growth_est_df = si.get_analysts_info(TICKER)['Growth Estimates']
growth_str = growth_est_df[growth_est_df['Growth Estimates'] == 'Next 5 Years (per annum)'][TICKER].iloc[0]
growth_rate = round(float(growth_str.rstrip('%')) / 100.0, 4)

In [5]:
print(f'Growth rate: {growth_rate}')
print(growth_str)

Growth rate: 0.0614
6.14%


In [11]:
PROJECTION_YEARS = 5
WACC_RATE = 0.094
PERP_GROWTH_RATE = 0.025

In [24]:
# Using yfinance to get other data
ticker = yf.Ticker(TICKER)
 
# Free cash flow
free_cash_flow = ticker.cash_flow.loc['Free Cash Flow'][0]

ffcf = []
# Year 1
ffcf.append(free_cash_flow * (1 + growth_rate))

# Starting from year 2
for i in range(1, PROJECTION_YEARS):
    ffcf.append(ffcf[i-1] * (1 + growth_rate))

print(ffcf)

[105698457599.99998, 112188342896.63997, 119076707150.49365, 126388016969.53395, 134148241211.46332]


In [12]:
# forecast_fcf[-1] refers to the last year in the growth period
terminal_value = ffcf[-1] * (1 + PERP_GROWTH_RATE)/(WACC_RATE - PERP_GROWTH_RATE)

In [16]:
stats = pd.DataFrame(columns=['Year', 'Free Cash Flow'])
discount_factor = 0.3
stats['Year'] = range(1, PROJECTION_YEARS + 1)
stats['Free Cash Flow'] = ffcf

print(stats.round(2))

   Year  Free Cash Flow
0     1    1.056985e+11
1     2    1.121883e+11
2     3    1.190767e+11
3     4    1.263880e+11
4     5    1.341482e+11


In [18]:
# Enterprice value

# Add the terminal value to the last year
ffcf[-1] = ffcf[-1] + terminal_value

# Calculate dcf using npv - add zero or else the method assumes
# first value as the initial investment
enterprise_value = npf.npv(WACC_RATE, [0] + ffcf)

print(f'Enterprise value: {enterprise_value}')

Enterprise value: 2998469609333.2695


In [19]:
# Calculate Cash And Cash Equivalents
balance_sheet_df = ticker.balance_sheet
cash_and_equivalents = balance_sheet_df.loc['Cash And Cash Equivalents'][0]

# Current debt - only interested in the value
current_debt = balance_sheet_df.loc['Current Debt'][0]

# Long term debt - only interested in the value
long_term_debt = balance_sheet_df.loc['Long Term Debt'][0]

# Equity value -> enterprise_value + cash - total debt
equity_value = enterprise_value + cash_and_equivalents - (current_debt +  long_term_debt)

In [43]:
something = si.get_income_statement(ticker=TICKER)

print(something)

AAPL


TypeError: string indices must be integers

In [21]:
# Shares outstanding
shares_outstanding = pd.to_numeric(overview['SharesOutstanding'])

# Current stock price
current_value = si.get_live_price(TICKER)

# Intrinsic value
intrinsic_value = equity_value / shares_outstanding
'Current Price: ${:.2f} Intrinsic Value: ${:.2f}'.format(round(current_value,2), round(intrinsic_value,2)) + ' for ticker ' + TICKER

HTTPError: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v6/finance/quoteSummary/AAPL?modules=financialData&modules=quoteType&modules=defaultKeyStatistics&modules=assetProfile&modules=summaryDetail&ssl=true